### Crawling Berita

Crawling berita adalah proses mengambil atau mengekstrak informasi berita dari situs web secara otomatis dengan menggunakan bot atau skrip komputer. Tujuan dari crawling berita adalah untuk mengumpulkan data berita dari berbagai sumber secara efisien, terstruktur, dan otomatis.

## Import Drive

Fungsi drive.mount('/content/drive') ini memberikan akses ke Google Drive Anda dalam lingkungan Google Colab, dan direktori Drive akan muncul di lokasi yang disebutkan, yaitu /content/drive. Ini memungkinkan kita menyimpan dan mengakses file langsung dari atau ke Google Drive selama sesi Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Library

1. requests: Library untuk mengirim permintaan HTTP dan menerima respons. Dengan menggunakan requests, Anda dapat mengambil konten dari suatu URL.

2. BeautifulSoup (bs4): Library untuk scraping data dari HTML dan XML. BeautifulSoup mempermudah ekstraksi informasi dari halaman web dengan menyediakan antarmuka Python yang bersih.

3. csv: Library untuk membaca dan menulis file CSV. Dengan menggunakan csv, Anda dapat menyimpan atau memuat data dalam format CSV.

4. pandas: Library untuk analisis data dan manipulasi data. pandas menyediakan struktur data seperti DataFrame, yang memudahkan pengolahan dan analisis data.






In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd


Menentukan kategori apa saja yang ingin di crawling dan jumlah data per kategorinya

In [ ]:
# Daftar kategori yang dapat diisi custom sesuai kebutuhan
kategori = ['Pemilu', 'Sport', 'Lifestyle']

# Jumlah data yang diambil per kategori
jumlah_data_per_kategori = 200

## Proses Crawling Berita

1. data_list = [] digunakan untuk menyimpan data berita sebelum dimasukkan ke dalam DataFrame

2. Looping Kategori dan Halaman

  - #Looping pertama : untuk mengiterasi melalui setiap kategori dan halaman pencarian situs berita

  - #Akses dan scraping halaman pencarian : Kode ini mengakses halaman pencarian, mengambil HTML-nya, dan menggunakan BeautifulSoup untuk mengekstrak elemen-elemen yang mengandung berita.

  - #Looping untuk setiap berita dalam halaman tertentu : ini mengiterasi melalui setiap elemen berita pada halaman dan mengekstrak judul, tanggal, dan link berita.

  - #Looping link berita untuk diambil data kontennya : ini mengakses link berita untuk mendapatkan konten berita.

  - #Menambahkan data ke dalam list : Setiap data berita yang telah diambil disimpan dalam bentuk dictionary dan ditambahkan ke dalam list data_list.

  - #Membuat dataframe dari list data : List data kemudian digunakan untuk membuat DataFrame menggunakan pandas, dan DataFrame tersebut disimpan ke dalam file CSV dengan path lokal.

  -


In [3]:
# List untuk menyimpan data
data_list = []

## Looping 1
# Looping untuk setiap kategori
for kategori_terpilih in kategori:
    katakunci = kategori_terpilih
    # Looping untuk setiap halaman
    for halaman in range(1, jumlah_data_per_kategori // 20 + 1):  # 20 berita per halaman
        ##Akses dan scraping halaman pencarian
        url = "https://www.beritasatu.com/search/{}/{}".format(katakunci, halaman)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0'
        }
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        items = soup.findAll('div', 'row mt-4 position-relative')

        ## Looping untuk setiap berita dalam halaman tertentu
        for it in items:
            # Looping judul
            judul = it.find('h2', 'h5 fw-bold').text.strip()
            # Looping tanggal berita
            tanggal_tag = it.find('span', 'b1-date text-muted')
            tanggal = ''
            if tanggal_tag:
                tanggal = ' '.join(tanggal_tag.text.strip().split('\n'))

            ## Looping link berita untuk diambil data kontennya
            link = it.find('a', 'stretched-link')
            if link:
                href = link.get('href')
                response = requests.get(href, headers=headers)
                berita_soup = BeautifulSoup(response.text, 'html.parser')
                #Looping untuk konten berita jika link bisa diakses
                konten_tag = berita_soup.find('div', 'col b1-article body-content')
                konten = ''
                if konten_tag:
                    konten = konten_tag.text.strip()

                ## Menambahkan data ke dalam list
                data_list.append({'Kategori': katakunci, 'Judul': judul, 'Tanggal': tanggal, 'Link': href, 'Konten': konten})

## Membuat dataframe dari list data
df = pd.DataFrame(data_list)

# Menyimpan dataframe ke dalam file CSV dengan path lokal
file_path = 'Crawling-DataBerita.csv'
df.to_csv(file_path, index=False)

print("Pengambilan data selesai.")

Pengambilan data selesai.


## Menampilkan Data yang berhasil di Crawling

In [4]:
df

,Kategori,Judul,Tanggal,Link,Konten
0,Pemilu,Aprindo: Pesta Demokrasi Dorong Pertumbuhan Ek...,7 jam yang lalu,https://www.beritasatu.com/ekonomi/2784459/apr...,"Jakarta, Beritasatu.com - Ketua Umum Asosiasi ..."
1,Pemilu,Ratusan Kotak Suara Pemilu di Jember Ditemukan...,10 jam yang lalu,https://www.beritasatu.com/nusantara/2784421/r...,"Jember, Beritasatu.com - Komisi Pemilihan Umum..."
2,Pemilu,11.000 Personel Polairud Amankan Pemilu 2024,10 jam yang lalu,https://www.beritasatu.com/bersatu-kawal-pemil...,"Jakarta, Beritasatu.com - Kepala Badan Pemelih..."
3,Pemilu,Kenali Perbedaan Alat Peraga Kampanye dan Sosi...,12 jam yang lalu,https://www.beritasatu.com/bersatu-kawal-pemil...,"Jakarta, Beritasatu.com – Saat ini Pemilihan U..."
4,Pemilu,TKN: Paslon Lain Mulai Mengapresiasi Gaya Poli...,13 jam yang lalu,https://www.beritasatu.com/bersatu-kawal-pemil...,"Jakarta, Beritasatu.com - Komandan Tim Komunik..."
...,...,...,...,...,...
595,Lifestyle,"Raih Medali di Hanoi, DG Reeva Ingin Bawa Espo...",26 Mei 2022 | 23:15 WIB,https://www.beritasatu.com/sport/932073/raih-m...,"Jakarta, Beritasatu.com – Atlet DG Esports Dun..."
596,Lifestyle,"Genjot Pemasaran, Synthesis Huis Gandeng Tiga ...",26 Mei 2022 | 19:37 WIB,https://www.beritasatu.com/ekonomi/931987/genj...,"Jakarta, Beritasatu.com - PT Synthesis Karya P..."
597,Lifestyle,"Pelonggaran Perjalanan, Traveloka Catat Kenaik...",25 Mei 2022 | 12:56 WIB,https://www.beritasatu.com/lifestyle/931395/pe...,"Jakarta, Beritasatu.com - Seiring dengan kebij..."
598,Lifestyle,Halal Industri Event 2022,24 Mei 2022 | 19:41 WIB,https://www.beritasatu.com/photo/82289/halal-i...,Wapres KH. Ma’ruf Amin mengatakan bahwa kolabo...


## Menampilkan Jumlah Data dalam kategori

1. Menggunakan len dan df[df['Kategori'] == ...]: untuk menghitung jumlah baris (data) yang memiliki nilai tertentu di kolom 'Kategori'.
2. print : mencetak jumlah data untuk masing-masing kategori.

In [7]:
# Menampilkan jumlah data untuk kategori tertentu
jumlah_sport = len(df[df['Kategori'] == 'Sport'])
jumlah_pemilu = len(df[df['Kategori'] == 'Pemilu'])
jumlah_lifestyle = len(df[df['Kategori'] == 'Lifestyle'])

# Menampilkan hasil
print(f"Jumlah data untuk Sport: {jumlah_sport}")
print(f"Jumlah data untuk Pemilu: {jumlah_pemilu}")
print(f"Jumlah data untuk Lifestyle: {jumlah_lifestyle}")


Jumlah data untuk Sport: 200
Jumlah data untuk Pemilu: 200
Jumlah data untuk Lifestyle: 200
